Naive-Bayes Algorithm
=====================
***

What Is It?
-----------

The Naive-Bayes algorithm is an intuitive approach to making predictions based on prior beliefs or probabilities. Quoting Jason Brownlee, "it is the supervised learning approach you would come up with if you wanted to model a predictive modeling problem probabilistically".

Let's dive into the mathematics. We start off with a belief or a *prior probability* of event $A$. This is denoted as $P(A)$. Now, everything seems to be going well until we're hit with some new evidence $X$, which implies something that affects the probability of our belief. As much as we'd like to, we can't simply ignore $X$ and go home. Instead, given evidence $X$, we must calculate a new value for event $A$ called the *posterior probability*. This is denoted as $P(A | X)$. Finally, for the sake of completion, $P(X | A)$ is the probability of observing evidence $X$ for event $A$ and $P(X)$ is the untouched probability of observing evidence $X$.

\begin{align}
 P( A | X ) = & \frac{ P(X | A) P(A) } {P(X) } \\\\[5pt]
\end{align}

You're probably wondering what makes this algorithm *naive*. Well, it's due to the underlying assumption that the probability of event $A$ given any evidence $X_n$ is totally independent of each other. This simplifies a lot of things and explains its popularity in many fields.

The content of this notebook uses Python to classify whether a patient is diagnosed with diabetes given a set of attributes. The data set is called the "Pima Indians Diabetes Data Set" provided by the National Institute of Diabetes and Digestive and Kidney Diseases. The target accuracy to indicate the algorithm's credibility is between 70% - 76%.

Data Loading and Formatting
-------------------------------

The data set is given as a `csv` file, which requires parsing and partitioning to form a training set and a test set.

In [1]:
import csv
def load_csv(file):
    lines = csv.reader(open(file, 'rb'))
    dataset = list(lines)
    for i in range(len(dataset)):
        dataset[i] = [float(x) for x in dataset[i]]
    return dataset

file = "pima-indians-diabetes.data.csv"
dataset = load_csv(file)
print('Loaded data from {0} with {1} rows').format(file, len(dataset))

Loaded data from pima-indians-diabetes.data.csv with 768 rows


In [2]:
from random import randrange
def partition_data(dataset, ratio):
    train_size = int(len(dataset) * ratio)
    test_set = list(dataset)
    train_set = []
    
    while len(train_set) < train_size:
        index = randrange(len(test_set))
        train_set.append(test_set.pop(index))
        
    return [train_set, test_set]

train_set, test_set = partition_data(dataset, 0.67)
print('Split total data ({0} rows) into training set ({1} rows) and testing set ({2} rows)').format(len(dataset), len(train_set), len(test_set))
    

Split total data (768 rows) into training set (514 rows) and testing set (254 rows)


Data Organization and Pre-calculations
--------------------------------------

Now that our dataset has been partitioned, let's visualize what it actually looks like and discuss how we should transform it going forward. Currently, our training set, $T$, can be described as an $m \times n$ matrix,

\begin{align}
T = 
\begin{bmatrix}
    x_{11}       & x_{12} & x_{13} & \dots & x_{1n} \\
    x_{21}       & x_{22} & x_{23} & \dots & x_{2n} \\
    \vdots & \vdots & \vdots & \ddots & \vdots \\
    x_{m1}       & x_{m2} & x_{m3} & \dots & x_{mn}
\end{bmatrix}
\end{align}

where $m$ is the number of data points and $n$ is the number of attributes including the classification value for each data point.

Next, we want to summarize our data. First, we want to group our data points by classification value, giving us two matrices, $T(0)$ and $T(1)$. Afterwards, we must calculate the mean and standard devation for each attribute and store them as a matrix of tuples in preparation for our probability function later on.

\begin{align}
T(0) = 
\begin{bmatrix}
    (\bar{x}, s)_{1} \\
    (\bar{x}, s)_{2} \\
    \vdots \\
    (\bar{x}, s)_{n}
\end{bmatrix}
\\
\\
T(1) = 
\begin{bmatrix}
    (\bar{x}, s)_{1} \\
    (\bar{x}, s)_{2} \\
    \vdots \\
    (\bar{x}, s)_{n}
\end{bmatrix}
\end{align}

In the implementation, we'll summarize the data set into a dict where each key and value are the classification value and attribute summaries, respectively.

In [3]:
def group_by_class(dataset):
    klass_map = {}
    for el in dataset:
        klass = int(el[-1])
        if klass not in klass_map:
            klass_map[klass] = []
        klass_map[klass].append(el[:-1])
    return klass_map

classified_set = group_by_class(train_set)

for klass, data_points in classified_set.iteritems():
    print('Class {0} contains {1} data points').format(klass, len(data_points))

Class 0 contains 325 data points
Class 1 contains 189 data points


We define our mean and standard deviation calculations as functions.

In [4]:
import math
def mean(n):
    return sum(n) / float(len(n))

def stdev(n):
    average = mean(n)
    return math.sqrt(sum([pow(x - average, 2) for x in n]) / float(len(n) - 1))

We can optimize our process time by parallelizing our mean and standard deviation calculations using the built-in multiprocessor library.

In [5]:
import multiprocessing as mp

def format_calc(t):
    return (mean(t), stdev(t))

def prepare_data(dataset):
    pool = mp.Pool(mp.cpu_count())
    summary = {}
    for klass, data_points in dataset.iteritems():
        summary[klass] = pool.map(format_calc, zip(*data_points))
    pool.close()
    pool.join()
    return summary

summary_set = prepare_data(classified_set)

for klass, tupl in summary_set.iteritems():
    print('Class {0} contains {1} tuples').format(klass, len(tupl))

Class 0 contains 8 tuples
Class 1 contains 8 tuples


Data Training and Testing
-------------------------

We can now create the Naive-Bayes classifier using our training set and a Guassian probability function to produce $P(A|X)$. This is the probability of an attribute belonging to a class and due to the assumption that each probability is independent from each other, the calculation process can be parallelized (for future implementation).

In [6]:
import math

def gauss(x, mean, stdev):
    ex = math.exp(-(math.pow(x - mean, 2) / (2 * math.pow(stdev, 2))))
    return (1 / (math.sqrt(2 * math.pi) * stdev)) * ex

In [7]:
def predict(summary_set, data_point):
    probabilities = {}
    for klass, summary in summary_set.iteritems():
        probabilities[klass] = 1
        for i in xrange(len(summary)):
            mean, stdev = summary[i]
            probabilities[klass] *= gauss(data_point[i], mean, stdev)
    return max(probabilities.iterkeys(), key=(lambda key: probabilities[key]))

Let's test our model against the training set and hope that the accuracy falls in the acceptable range.

In [8]:
def get_accuracy(summary_set, test_set):
    correct_count = 0
    for test_point in test_set:
        if test_point[-1] == predict(summary_set, test_point):
            correct_count += 1
    return correct_count / float(len(test_set)) * 100

accuracy = get_accuracy(summary_set, test_set)

print('The Naive-Bayes Model yields {0}% accuracy').format(round(accuracy, 2))

The Naive-Bayes Model yields 75.59% accuracy


Our accuracy lies between 70% - 76% range; therefore, our model is accurate (at least for the purpose of this notebook).